In [11]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
from autograd.test_util import check_grads

In [13]:
import pkg_resources
pkg_resources.get_distribution('autograd').version

import inspect
inspect.getsourcefile(autograd)

'/home/rgiordan/Documents/git_repos/autograd/autograd/__init__.py'

# Direct from github issue https://github.com/HIPS/autograd/issues/460

In [20]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
from autograd.test_util import check_grads

num_obs = 1000

# Some valid values
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Succeeds
print('Differentiation succeeds')
check_grads(autograd.scipy.stats.multivariate_normal.logpdf, modes=['rev'])(x, true_mu, true_sigma)

# Fails
try:
    def logpdf_mu(mu):
        return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mu, true_sigma))
    print('Can evaluate the function: ', logpdf_mu(true_mu))
    check_grads(logpdf_mu, modes=['rev'])(true_mu)
except AssertionError as err:
    print('Differentiation fails:\ntest_util.py 31:\tassert vspace(vjp_y) == x_vs')

try:
    def logpdf_mu(mu):
        return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mean=mu, cov=true_sigma))
    print('Can evaluate the function: ', logpdf_mu(true_mu))
    check_grads(logpdf_mu, modes=['rev'])(true_mu)
except TypeError as err:
    print('Differentiation fails: ' + str(err))

Differentiation succeeds
Can evaluate the function:  -5236.396067015383
Differentiation fails:
test_util.py 31:	assert vspace(vjp_y) == x_vs
Can evaluate the function:  -5236.396067015383
Differentiation fails: Can't differentiate w.r.t. type <class 'numpy.int64'>


In [22]:
def sym_logpdf(sigma):
    sigma_sym = 0.5 * (sigma + sigma.T)
    return np.sum(
        autograd.scipy.stats.multivariate_normal.logpdf(
            x, true_mu, sigma_sym))

try:
    check_grads(sym_logpdf, modes=['rev'])(true_sigma)
    print('Differentiation succeeds')
except AssertionError as err:
    print('Differentiation fails: {}'.format(err))

Differentiation succeeds


In [31]:
def model_logpdf(mu, sigma, x):
    sigma_sym = 0.5 * (sigma + sigma.T)
    data_lpdf = autograd.scipy.stats.multivariate_normal.logpdf(
        x, mu, sigma_sym)
    return np.sum(data_lpdf)

def model_logpdf_sigma(sigma):
    return model_logpdf(true_mu, sigma, x)

try:
    check_grads(model_logpdf_sigma, modes=['rev'])(true_sigma)
    print('Differentiation succeeds')
except AssertionError as err:
    print('Differentiation fails: {}'.format(err))

Differentiation succeeds


# Manual stuff below:

In [24]:
num_obs = 1000

# Some valid values
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0., 1., 2.])

x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Succeeds
print('Differentiation succeeds')
check_grads(autograd.scipy.stats.multivariate_normal.logpdf, modes=['rev'])(x, true_mu, true_sigma)

# Succeeds
try:
    def logpdf_mu(mu):
        return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mu, true_sigma))
    print('Can evaluate the function: ', logpdf_mu(true_mu))
    check_grads(logpdf_mu, modes=['rev'])(true_mu)
    print('Differentiation succeeds')
except AssertionError as err:
    print('Differentiation fails:\ntest_util.py 31:\tassert vspace(vjp_y) == x_vs')

    
def sym_logpdf(sigma):
    sigma_sym = 0.5 * (sigma + sigma.T)
    return np.sum(
        autograd.scipy.stats.multivariate_normal.logpdf(
            x, true_mu, sigma_sym))

try:
    print('Differentiation succeeds')
    check_grads(sym_logpdf, modes=['rev'])(true_sigma)
except AssertionError as err:
    print('Differentiation fails: {}'.format(err))

# try:
#     def logpdf_mu(mu):
#         return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mean=mu, cov=true_sigma))
#     print('Can evaluate the function: ', logpdf_mu(true_mu))
#     check_grads(logpdf_mu, modes=['rev'])(true_mu)
# except TypeError as err:
#     print('Differentiation fails: ' + str(err))




Differentiation succeeds
Can evaluate the function:  -5152.106572743378
Differentiation succeeds
Differentiation succeeds


In [25]:
# Asymmetric sigmas are allowed apparently.

import copy
import scipy as osp
bad_sigma = copy.deepcopy(true_sigma)
bad_sigma[1, 2] += 2
print(np.linalg.norm(bad_sigma - bad_sigma.T))

print(np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, true_mu, bad_sigma)))
print(np.sum(osp.stats.multivariate_normal.logpdf(x, true_mu, bad_sigma)))

2.8284271247461903
-5152.106572743378
-5152.106572743378


In [26]:
def sigma_logpdf(sigma):
    return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, true_mu, sigma))

# Taken from tests
def symmetrize_matrix_arg(fun, argnum):
    def T(X): return np.swapaxes(X, -1, -2) if np.ndim(X) > 1 else X
    def symmetrize(X): return 0.5 * (X + T(X))
    def symmetrized_fun(*args, **kwargs):
        args = list(args)
        args[argnum] = symmetrize(args[argnum])
        return fun(*args, **kwargs)
    return symmetrized_fun

sigma_logpdf_sym = symmetrize_matrix_arg(sigma_logpdf, 0)
sigma_logpdf_sym(bad_sigma) - \
    sigma_logpdf_sym(0.5 * (bad_sigma + bad_sigma.T))
    
check_grads(sigma_logpdf_sym, modes=['rev'])(true_sigma)

In [28]:
def sym_logpdf(sigma):
    sigma_sym = 0.5 * (sigma + sigma.T)
    return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, true_mu, sigma_sym))

check_grads(sym_logpdf, modes=['rev'])(true_sigma)

In [32]:
def model_logpdf(mu, sigma, x):
    sigma_sym = 0.5 * (sigma + sigma.T)
    data_lpdf = autograd.scipy.stats.multivariate_normal.logpdf(
        x, mu, sigma_sym)
    return np.sum(data_lpdf)

def model_logpdf_sigma(sigma):
    return model_logpdf(true_mu, sigma, x)

check_grads(model_logpdf_sigma, modes=['rev'])(true_sigma)

In [33]:
model_logpdf_params_mu_grad = autograd.grad(autograd.scipy.stats.multivariate_normal.logpdf, argnum=2)
print(model_logpdf_params_mu_grad(x[0, :], true_mu, true_sigma))

model_logpdf_params_mu_grad = autograd.grad(autograd.scipy.stats.multivariate_normal.logpdf, argnum=1)
print(model_logpdf_params_mu_grad(x[0, :], true_mu, true_sigma))

print(autograd.scipy.stats.multivariate_normal.logpdf(x[0, :], true_mu, true_sigma))
print(autograd.scipy.stats.multivariate_normal.logpdf(x[0, :], mean=true_mu, cov=true_sigma))

[[-0.16676418  0.00309269  0.14193447]
 [ 0.00309269 -0.24008802 -0.00058696]
 [ 0.14193447 -0.00058696 -0.10984043]]
[ 0.81563858 -0.00722529  0.32893988]
-4.193433622482122
-4.193433622482122


In [37]:
def model_logpdf(mu, sigma, x):
    sigma_sym = 0.5 * (sigma + sigma.T)
    data_lpdf = autograd.scipy.stats.multivariate_normal.logpdf(
        x, mu, sigma_sym)
    return np.sum(data_lpdf)

model_logpdf(true_mu, true_sigma, x)
check_grads(model_logpdf, modes=['rev'])(true_mu, true_sigma, x)


def model_logpdf_mu(mu):
    return model_logpdf(mu, true_sigma, x)
check_grads(model_logpdf_mu, modes=['rev'])(true_mu)

In [38]:
import paragami
mu_pattern = paragami.NumericVectorPattern(length=3)
objective_flat = paragami.FlattenFunctionInput(
    model_logpdf_mu,
    patterns=mu_pattern,
    free=True)

objective_flat(true_mu)
check_grads(objective_flat, modes=['rev'])(true_mu)


In [39]:
def model_logpdf_sigma(sigma):
    return model_logpdf(true_mu, sigma, x)

check_grads(model_logpdf_sigma, modes=['rev'])(true_sigma)